In [ ]:
import numpy as np
import pandas as pd

x_pos = np.arange(1, 1001)
x_neg = -x_pos

df_pos = pd.DataFrame({
    "x": x_pos,
    "exp_x": np.exp(np.arange(1, 1001)) # will likely overflow
})

df_neg = pd.DataFrame({
    "x": x_neg,
    "exp_x": np.exp(-np.arange(1, 1001)) # will likely underflow
})

df_pos.head()

In [ ]:
df_neg.head()

In [ ]:
import numpy as np
import pandas as pd

# Force float64 explicitly
x_pos = np.arange(1, 1001, dtype=np.float64)
x_neg = -x_pos

df_pos_f64 = pd.DataFrame({
    "x": x_pos,
    "exp_x": np.exp(x_pos)
})

df_neg_f64 = pd.DataFrame({
    "x": x_neg,
    "exp_x": np.exp(x_neg)
})

df_pos_f64.head()

In [ ]:
df_neg_f64.head()

In [ ]:
overflow_idx = np.where(np.isinf(df_pos_f64["exp_x"]))[0][0]
underflow_idx = np.where(df_neg_f64["exp_x"] == 0.0)[0][0]

overflow_x = df_pos_f64.loc[overflow_idx, "x"]
underflow_x = df_neg_f64.loc[underflow_idx, "x"]

overflow_x, underflow_x

# Provided Utility Functions for Use in Homework

In [ ]:
import utilities as ut # looks for utilities.py in the same folder
dir(ut) # what named things get imported?

In [ ]:
X_syn, y_syn, w_true, b_true = ut.make_synthetic_linear(n=2000, d=8, noise_sd=1.0, seed=7)
(X_syn, y_syn, w_true, b_true)

# Examples to Guide Homework

In [ ]:
import numpy as np
import pandas as pd
import utilities as ut

In [ ]:
seed = 42

# Load data
df = pd.read_csv("data/auto-mpg.csv")

feature_cols = ["cylinders", "displacement", "horsepower", "weight"]
target_col = "mpg"

# Keep only rows with needed numeric data
df1 = df[feature_cols + [target_col]].dropna()

X = df1[feature_cols].to_numpy(dtype=np.float64)
y = df1[target_col].to_numpy(dtype=np.float64)

# Split
(train_X, train_y), (val_X, val_y), (test_X, test_y) = ut.train_val_test_split(X, y, seed=seed)

# Train with SGD
w, b, hist = ut.sgd_linear_regression(
    train_X, train_y,
    X_val=val_X, y_val=val_y,
    lr=1e-6,          # without normalization, LR usually needs to be small
    epochs=500,
    batch_size=16,
    seed=seed,
    verbose_every=50
)

print("\nFinal evaluation:")
print("Train:", ut.evaluate_linear_model(train_X, train_y, w, b))
print("Val:  ", ut.evaluate_linear_model(val_X, val_y, w, b))
print("Test: ", ut.evaluate_linear_model(test_X, test_y, w, b))

hist.tail()

In [ ]:
seed = 42

# Reload and clean (same as Part 1)
df = pd.read_csv("data/auto-mpg.csv")
df3 = df[feature_cols + [target_col]].dropna()

X = df3[feature_cols].to_numpy(dtype=np.float64)
y = df3[target_col].to_numpy(dtype=np.float64)

# Split FIRST (important: fit scaler only on train)
(train_X, train_y), (val_X, val_y), (test_X, test_y) = ut.train_val_test_split(X, y, seed=seed)

# Normalize using train statistics
mu = train_X.mean(axis=0)
sigma = train_X.std(axis=0, ddof=0)
sigma = np.where(sigma == 0, 1.0, sigma)  # safety

train_Xn = (train_X - mu) / sigma
val_Xn   = (val_X - mu) / sigma
test_Xn  = (test_X - mu) / sigma

# Train with a larger LR now that scales are consistent
w_n, b_n, hist_n = ut.sgd_linear_regression(
    train_Xn, train_y,
    X_val=val_Xn, y_val=val_y,
    lr=1e-2,
    epochs=300,
    batch_size=16,
    seed=seed,
    verbose_every=25
)

print("\nFinal evaluation (normalized features):")
print("Train:", ut.evaluate_linear_model(train_Xn, train_y, w_n, b_n))
print("Val:  ", ut.evaluate_linear_model(val_Xn, val_y, w_n, b_n))
print("Test: ", ut.evaluate_linear_model(test_Xn, test_y, w_n, b_n))

hist_n.tail()